In [1]:
import sys
import stmpy
import numpy as np
import cancelv3 as cancel
from decimal import *
import matplotlib.pyplot as plt
from sklearn.externals import joblib
import pandas as pd
import scipy.fftpack as fftp
import os
from scipy import signal
import scipy
import matplotlib.pyplot as plt
import time
from mpl_toolkits.axes_grid1.inset_locator import inset_axes, mark_inset
%pylab inline

C:\Users\jgoet\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


Populating the interactive namespace from numpy and matplotlib


In [2]:
# Functions used in generating synthetic environmental data
#     create_wave will call the previous functions and create a waveform 
#     that incorporates random phase and amplitude changes over time

#designate randomly selected points of time that have a phase slip 
def slipper(t_array,num, debug = False):
    #num: how many phase slips should be incorporated into the frequency
    #indexInt = random.randint(0,int(len(sz)/rate))
    ti = random.choice(arange(len(t_array)),num)
    return ti

#create Phase Map
def phase_map(t_array, loc,debug = False):
    slipSz = t_array.shape
    loc = np.sort(loc)
    PhaseMap = np.ones(slipSz)*(2*np.pi*random.rand(1)-np.pi)
    for point in loc:
        PhaseMap[point:] = 2*np.pi*random.rand(1)-np.pi
    return PhaseMap

#create Amplitude Map
def amp_map(t_array, slip_loc, freq, amp_i=1, mod=0.1, mod_amp=0.05,debug = False):
    #t_array: the time series values
    #          t_array[1] is used to find the sample rate that was used
    #loc: the index of random time points that a phase shift occurrs
    #freq: frequency that is being modeled. Gaussian width is dependant on freq
    #res_width: resonance width, the bandwidth over which the power of vibration is greater than 
    #               half the power at the resonant frequency
    #gauss_width: width of gaussian curve such that frequency decays to zero at 2 periods
    #amp_i: initial amplitude of function
    #mod: frequency of amplitude madulation
    #mod_amp: amplitude of amplitude modulation
    #debug: for debugging purposes
    
    period = 1/(freq)
    gauss_width = int(2*period*(1/t_array[1]))
    
    res_width = 0.0001
    res_freq = 1
    Q_factor = res_freq/res_width
    
    dropSz = t_array.shape
    slip_loc = np.sort(slip_loc)
    
    variance = 0.1*amp_i
    t1 =time.time()
    AmpMap = np.ones(dropSz)*amp_i + random.normal(0,variance,1)
    t2 = time.time()
    if debug:
        AmpMap2 = np.copy(AmpMap)
    for point in slip_loc:
        change_up = random.normal(0,variance,1)
        if debug:
            AmpMap2[point:] = 1*amp_i + change_up
        AmpMap[point:] = (1*amp_i + change_up)*exp(-(1/Q_factor)*(t_array[point:]-t_array[point]))
        AmpMap[int(point-gauss_width/2):int(point+gauss_width/2)] = 0
    t3 = time.time()
    
    window = signal.hamming(int(gauss_width))
    AmpMap = signal.convolve(AmpMap,window, mode='same')/sum(window)
    AmpMap  = AmpMap #+ mod_amp*sin(2*pi*mod*t_array)
    if debug:
        return AmpMap,AmpMap2
    return AmpMap

def noisy(t_array, amp, fs = 10000, cutoff = 1000):
    noise = amp*randn(len(t_array))
    sos = signal.butter(10, cutoff, 'low', fs = fs, output = "sos")
    filtered = signal.sosfilt(sos, noise)
    return filtered

def create_wave(freq_array, amp_array, t_array = None, secs = 10, fs = 10000, slips = 10,noise = True):
    
    if t_array == None:
        t_array = linspace(0,secs,secs*fs)
        
    if noise:
        low_noise = noisy(t_array,0.000005)
    else:
        low_noise = 0
    tempWave = np.zeros(len(t_array)) + low_noise
    for freq,ampi in zip(freq_array,amp_array):
        ampi = ampi/40000
        
        ti = slipper(t_array,slips)
        phase = phase_map(t_array,ti)
        amp = amp_map(t_array,ti,freq,amp_i = ampi)
        tempWave += amp*sin(2*np.pi*freq*t_array + phase)
    
    return tempWave, t_array


In [3]:
#Functions for creating a stepwise frequency chirp
# that does not abruptly just between frequencies
# It drops to 0 with each frequency change over a 
#  set number of periods to avoid high frequency noise
def chirp_amp_map(freq_map,t_array):
    
    period = 1/(freq_map)
    gauss_width = 2*period*(1/t_array[1])
    
    res_width = 0.0001
    res_freq = 1
    Q_factor = res_freq/res_width
    
    dropSz = len(t_array)
    
    loc = np.where(freq_map[:-1] != freq_map[1:])[0]

    gauss_width = gauss_width[loc]
    AmpMap = np.ones(dropSz)
    
    for point,width in zip(loc,gauss_width):
        width = int(np.around(width))
        
        point = point + width #width acts as buffer for the convolution to not affect the ends of the array
        
        tempMap = np.ones(int(dropSz+2*width))
        start = int(point-width/2)
        end = int(point+width/2)
        tempMap[start:end] = 0
        
        
        window = signal.hamming(width)
        tempMap = signal.convolve(tempMap,window,mode='same')/sum(window)
        
        AmpMap = AmpMap*tempMap[width:-width]

    return AmpMap,loc

#Time at each frequency is equal to the designated number of periods
#   Output is a waveform of the chirp and the determined time
def chirp_wave(period = 10, res=1,final_freq=300,freq_time = 100, timer=False,chirp_type='linear'):
    
    if timer:
        t1 = time.time()
    final_freq = final_freq+res
    freq_base = linspace(res,final_freq,int((final_freq-res)/res))
    
    if chirp_type == 'exponential':
        t_len = sum(period/freq_base)
        peat_num = np.around((period/freq_base)*10000).astype('int')
        freq_vec = repeat(freq_base,peat_num)
    if chirp_type == 'linear':
        t_len = (final_freq/res)*freq_time
        peat_num = np.around((t_len/final_freq)*10000).astype('int')
        freq_vec = repeat(freq_base,peat_num)
        
    t_vec = linspace(0,t_len,t_len*10000)

    

    vec_dif = int(len(t_vec)-len(freq_vec))
   
    if vec_dif > 0:   #time vector is bigger take some off end
        t_vec = t_vec[:-vec_dif]
    elif vec_dif < 0:   #freq vector is bigger take some off beginning
        freq_vec = freq_vec[absolute(vec_dif):]
    else:
        pass
     
    amp,loc = chirp_amp_map(freq_vec,t_vec)
    
    chirp = amp*np.sin(2*np.pi*freq_vec*t_vec)
    
    #plot(t_vec,chirp)
    #plot(t_vec,amp)
    # plot(t_vec[loc],chirp[loc],'.')
    
    if timer:
        print('Chirp generation time: ',np.round(time.time()-t1,2),'sec')
        
    return chirp,t_vec,loc,freq_base

In [1]:
figsize(15,6)
chirp,t_vce,loc,freq_base = chirp_wave(final_freq=2)
plot(t_vce,chirp)

NameError: name 'figsize' is not defined

In [5]:
def butterworth_lowpass(sig, cutoff, fs = 10000):
    sos = signal.butter(8, cutoff, 'lowpass', fs=fs, output='sos')
    filtered = signal.sosfilt(sos, sig)
    return filtered

def geophone_response(sig, cutoff, fs = 10000, lor_width = 20):
    b,a = signal.butter(sig,cutoff, 'hp', fs = fs)
    w,h = signal.freqz(b,a)
    plt.semilogx(w, 1000*lorentzian(w,cutoff,lor_width)+ 20 * np.log10(abs(h)))
    filtered = 1000*lorentzian(w,cutoff,lor_width)+ 20 * np.log10(abs(h))

def resonant_filter(sig, w_res, res_width, fs = 10000):
    fft_sig = fft.fft(sig)
    fft_freq = fft.fftfreq(len(sig),1/fs)
    sos = signal.butter(8, 12, 'highpass', fs=fs, output='sos')
    filt_freq, filt_response = signal.sosfreqz(sos,worN=len(fft_sig),fs=fs)
    lor_response = lorentzian(fft_freq,w_res,res_width)
    res_response = 1*lor_response+(20*np.log10(absolute(filt_response)))
    res_response[int(len(res_response)/2)+1:] = np.flip(res_response[1:int(len(res_response)/2)])
    fft_filt = fft_sig*exp(res_response)
    fft_filt[0] = 0+0j
    return fft.ifft(fft_filt)

def butterworth_highpass(sig, cutoff, fs = 10000):
    sos = signal.butter(10, cutoff, 'hp', fs=fs, output='sos')
    filtered = signal.sosfilt(sos, sig)
    return filtered

def lorentzian(w, w0, g):
    return (1/pi) * (g/2) / ( (w-w0)**2 + (g/2)**2 )

def create_drive(zdata, dt2, tf, freqs):
    fbottom2 = np.fft.fft(zdata)
    freq2 = np.fft.fftfreq(len(zdata),dt2)
    freq2[freq2 > freqs[np.argmax(freqs)]] = freqs[np.argmax(freqs)]
    freq2[freq2 < freqs[np.argmin(freqs)]] = freqs[np.argmin(freqs)]
    
    driver = np.fft.ifft(tf(freq2)*fbottom2)
    return driver

### Environmental Vibrations

In [ ]:
figsize(15,6)
#frequencies and magnitudes found upon inspection of data from 2019-02-21
freqs = [1.6, 2, 2.5, 30, 33, 34, 36, 50, 57, 59.5, 100, 101, 111,
         118, 125, 263, 264, 265, 266, 267, 270, 2000,2222,2355, 2365,2375,3955,3900,3950,3955,3951,3926,3989]
amps = [0.25, 0.3, 0.28, 0.25, 0.3, 0.12, 0.1, 0.1, 0.2, 0.04, 0.035, 0.04, 0.03,
        0.03, 0.02, 0.02, 0.015, 0.02, 0.01, 0.015, 0.01,0.13,0.16,0.12,0.1,0.14,0.1,0.1,0.13,0.11,0.12,0.1,0.1]
start = time.time()
wave,t = create_wave(freqs, amps,secs = 100,slips = 50)
print(time.time()-start,end= '\r')

In [ ]:
plot(t,wave)
xlim(0,10)
title('Synthetic Environmental Data')

### Geophone Data

In [ ]:
#put data through a high pass filter
# 
noise_freq = [0.1, 0.5, 1.8, 2.6, 3.1,5.7,6.1,16,25]
noise_amp = [0.25,0.2,0.5,0.425,0.15,0.05,0.1,0.02]
electric_freq = [60,120,180,240,300,360,420,480,540,600,660,720,780,840,900]
electric_amp =  [0.02,0.035,0.2,0.03,0.075,0.025,0.15,0.01,0.04,0.025,0.225,0.2,0.225,0.15,0.03]
# start = time.time()
geo = resonant_filter(wave,30,6)
geo1,t = create_wave(noise_freq, noise_amp,secs = 100,slips = 10) 
geo = geo+geo1
# print(time.time()-start)
elec_wave,t = create_wave(electric_freq,electric_amp,secs = 100,slips=0,noise=False)
geo = geo + elec_wave
fft_geo = np.fft.fft(geo)
freq_geo = np.fft.fftfreq(len(geo),1/10000)
semilogy(freq_geo[:int(len(fft_geo)/2)],abs(fft_geo[:int(len(fft_geo)/2)]/100))
# plot(t,wave,label='Environmental Vibrations')
# plot(t,geo,label='Geophone Response')
xlim(0,800)
ylim(1e-5,0.5)
title('Synthetic Geophone Frequency Response')
# print(fft_geo[1],type(fft_geo[1]))
# legend()

## Tip Data

In [ ]:
#load real, calibrated transfer function
[transf_Alb, freqs_Alb] = joblib.load('Albert_Data/2019-02-26/trx2_v2')

transf = transf_Alb
freqs = freqs_Alb
lnfrq = int(len(freqs)/2)

# print(freqs[1])
#create log spaced 
w_log = logspace(log10(freqs[1]),log10(freqs[-1+int(len(freqs)/2)]),100000)

T_log = transf(w_log) #transf is already an interpolation of frequencies

window = signal.hamming(int(300))
T_flt = signal.convolve(T_log,window, mode='same')/sum(window)

# plot(w_log,absolute(T_log))
# plot(w_log,absolute(T_flt))


f = scipy.interpolate.interp1d(w_log,T_flt, kind='linear')

w_lin = linspace(w_log[0],w_log[-1],lnfrq-1)
transf_flt = np.zeros(shape(freqs),dtype=np.complex128)

transf_flt[1:lnfrq] = f(w_lin)
transf_flt[lnfrq:] = np.flip(transf_flt[:lnfrq])

interp_transf = scipy.interpolate.interp1d(freqs,transf_flt, kind='linear')
phase = angle(transf(freqs))
# plot(freqs[:lnfrq],phase[:lnfrq])
plot(freqs[:lnfrq],absolute(transf_flt[:lnfrq]))
# plot(freqs,abs(transf_flt))
xlim(0,200)

In [ ]:
# Subtract mean and divide by range.
def normalize2(array):
    return (array-mean(array))/(max(array) - min(array))

#Transient Convolution
#NOPE 
# t = linspace(0,10,100000)
# t2 = linspace(0,10,100000)
# t2[:int(len(t2)/2)] = 0
# x = np.sin(10*2*np.pi*t) + 0.5*np.sin(50*2*np.pi*t)+ np.sin(60*2*np.pi*t2)
# freq = np.fft.fftfreq(len(x),1/10000)
# transf1 = interp_transf(freq)
# transf_wave = np.fft.ifft(transf1)
# transf_wave2 = np.fft.ifft(transf(freq))
# # plot(t,transf_wave2)
# # plot(t,transf_wave)
# y1 = create_drive(x,1/10000,interp_transf,freq)
# y2 = signal.convolve(x,transf_wave, mode='same')
# plot(t,normalize2(y1))
# # plot(t,normalize2(y2))
# plot(t,normalize2(x))
# # xlim(0,1)

In [ ]:
chirp, t_chirp,loc,freq_base = chirp_wave(period=20,timer=True)
print(loc[0])

In [ ]:
freq1 = chirp[loc[98]:loc[99]]
t1 = t_chirp[loc[3]:loc[4]]
freq1_fft = np.fft.fft(freq1)/len(freq1)
freq_freq = np.fft.fftfreq(len(freq1),1/10000)

# plot(freq_freq[0:1000],abs(freq1_fft[0:1000]))
# plot(t_chirp,chirp)
# xlim(0,300)
i = 0
for point in loc:
    chirp_part = np.zeros(len(chirp))
    if i == 0:
        chirp_part[0:point] = chirp[0:point]
        i=1
    else:
        chirp_part[prev_point:point] = chirp[prev_point:point]
#     plot(t_chirp,chirp_part)
    freq1_fft = np.fft.fft(chirp_part)/len(chirp_part)
    freq_freq = np.fft.fftfreq(len(chirp_part),1/10000)
    plot(freq_freq[:int(len(freq1_fft)/2)],abs(freq1_fft[:int(len(freq1_fft)/2)]))
    prev_point = point
    

xlim(0,200)


In [ ]:
plot(t_chirp,chirp)

In [ ]:
def create_tip_data_3d(g_dat, transfer_function, freqs,harmonic_freqs, decay = 0.05, lockin = 1250):
    tf = transfer_function(freqs)
    fft_g = fft.fft(g_dat)
    lenfrq = int(len(fft_g)/2)
    lock_index = (np.abs(freqs - lockin)).argmin()
    tip_dat = np.zeros(shape(g_dat),dtype=complex128)
    fft_harm = np.zeros((20,len(fft_g)),dtype=complex128)
    transf_mag = np.zeros((20,len(fft_g)),dtype=complex128)
    transf_index = np.zeros((20,len(fft_g)))
    for w in harmonic_freqs:
        w = (np.abs(freqs - w)).argmin()
        harmonics = [int(w*3),int(w*5),int(w*7),int(w*9),int(w*11),int(w*13),int(w*15),int(w*17),int(w*19),int(w*21),
                    int(w*23),int(w*25),int(w*27),int(w*29),int(w*31),int(w*33),int(w*35),int(w*37),int(w*39),int(w*41)]
#         amps = [decay**1,decay**2,decay**3,decay**4,decay**5,decay**6,decay**7,decay**8,decay**9,decay**10,
#                decay**11,decay**12,decay**13,decay**14,decay**15,decay**16,decay**17,decay**18,decay**19,decay**20]
        amps = [1/3,1/5,1/7,1/9,1/11,1/13,1/15,1/17,1/19,1/21,1/23,1/25,1/27,1/29,1/31,1/33,1/35,1/37,1/39,1/41]
        transf_index[:,w]= np.asarray(harmonics)
        
        transf_mag[:,w] = np.asarray(amps)*fft_g[w]
        
    transf_index = transf_index.astype(int)
    count = 1
    for a in arange(len(transf_index[0,:])):
        neg = (1)**count
        for b in arange(len(transf_index[:,0])):
            index = transf_index[b,a]
            mag = transf_mag[b,a]
            if index > lenfrq:
                if index < len(tf):
                    index = len(tf) - index
                else:
                    index,mag = 0,0
            fft_harm[b,index] = complex(mag.real,neg*mag.imag)
#             print(mag)
#             fft_harm[b,index] = mag
        count = count+1
#     for index,mag in zip(transf_index.flatten(),transf_mag.flatten()):
#         if index > lenfrq:
#             if index < len(tf):
#                 transf_index[count] = len(tf) - index
#             else:
#                 transf_index[count],transf_mag[count] = 0,0
    fft_harm[:,lenfrq+1:] = np.flip(-fft_harm[:,1:lenfrq],axis=1)
    terp_tran_1 = scipy.interpolate.interp1d(freqs,tf, kind='linear')
    
    temp_tip = create_drive(g_dat,1/10000,terp_tran_1,freqs)
#     print(len(temp_tip),len(g_dat))
    for c in arange(20):
#         print(len(fft.ifft(fft_harm[c,:])))
        temp_tip = temp_tip + fft.ifft(fft_harm[c,:])
        
#     temp_fft = fft.fft(temp_tip)
#     temp_fft[lock_index:2*lock_index] = 1e-1*temp_fft[:lock_index]
#     temp_fft[:lock_index]
    
    tip_dat = temp_tip
    return tip_dat,terp_tran_1

def chirped_tip(chirp_wave, geo_wave, t_array, freq_val, wave_loc,transfer_function,
                harmony = 'square', chirp_type = 'linear',num = 5):
    #input is chirp and geophone data seperately
    #=======================================================================================================
    #=======================Add harmonics to the existing waveform =========================================
    #=======================================================================================================
    wave = chirp_wave+geo_wave
    
    if harmony == 'square':
        harmonics = arange(int(num-1))*2+3
        amps = 1/harmonics
    i=1
    prev_point = 1000000
    for val_pnt,loc_pnt in zip(freq_val,wave_loc):
        temp_t = np.copy(t_array)
        chirp_part = np.zeros(len(wave))
        if i == 0:
            chirp_part[0:loc_pnt]= chirp_wave[0:loc_pnt]
            temp_t[loc_pnt:] = 0 
            i=1
        else:
            chirp_part[prev_point:loc_pnt] = chirp_wave[prev_point:loc_pnt]
            temp_t[loc_pnt:] = 0
            temp_t[:prev_point]=0
        harm_freq = harmonics*val_pnt
        chirp_mag = amps*max(chirp_part)
#         plot(t_array,chirp_part)
        for freq,mag in zip(harm_freq,chirp_mag):
            wave = wave + mag*np.sin(freq*2*np.pi*temp_t)
#             plot(t_array,wave)
        prev_point=loc_pnt
    #=======================================================================================================
    #=================Output of the original waveform plus added harmonics of chirp frequencies=============
    #=======================================================================================================
    
    #==========================Apply the standard transfer function to resulting wave=======================
    freqs = np.fft.fftfreq(len(wave),1/10000)
    chirp_tip = create_drive(wave,1/10000,transfer_function,freqs)
        
    return wave,chirp_tip
    
    
    

In [ ]:
t= linspace(0,20,200000)
t1= linspace(0,20,200000)
t_fin = linspace(0,20,200000)
print(len(t),len(t1),len(t_fin))
t[:140000] = 0
t[159999:] = 0
wave1 = np.sin(10*2*np.pi*t)
t1[:160000]= 0
t1[179999:]= 0
wave2 = np.sin(25*2*np.pi*t1)
freq_val = [10,25]
loc = [159999,179999]
wave = wave1+wave2
geo = 0.35*np.sin(2*2*np.pi*t_fin)+0.4*np.sin(3*2*np.pi*t_fin)+0.1*np.sin(1000*2*np.pi*t_fin)+0.3*np.sin(2.5*2*np.pi*t_fin)

transf_s = 0.5*np.ones(len(wave))
frq = np.fft.fftfreq(len(wave),1/10000)
transf_sample = scipy.interpolate.interp1d(frq,transf_s, kind='linear')


chirped_up,chirped_down = chirped_tip(wave,geo,t_fin,freq_val,loc,transf_sample)
# plot(t_fin,chirped_up)
# plot(t_fin,chirped_down)
# plot(t_fin,wave+geo)
# xlim(3.75,4.25)
# xlim(5.75,6.25)
freqs = [1.6, 2, 2.5, 30, 33, 34, 36, 50, 57, 59.5, 100, 101, 111,
         118, 125, 263, 264, 265, 266, 267, 270, 2000,2222,2355, 2365,2375,3955,3900,3950,3955,3951,3926,3989]
amps = [0.25, 0.3, 0.28, 0.25, 0.3, 0.12, 0.1, 0.1, 0.2, 0.04, 0.035, 0.04, 0.03,
        0.03, 0.02, 0.02, 0.015, 0.02, 0.01, 0.015, 0.01,0.13,0.16,0.12,0.1,0.14,0.1,0.1,0.13,0.11,0.12,0.1,0.1]
geo1,t = create_wave(freqs, amps,secs = 20,slips = 12)
wave_full = wave+geo1*10000
print(len(wave),len(geo),len(wave_full))
frq1= np.fft.fftfreq(len(wave_full[:140000]),1/10000)
frq2 = np.fft.fftfreq(len(wave_full[140000:]),1/10000)
fft_cal_geo = np.fft.fft(wave_full[:140000])/len(wave_full[:140000])
fft_full = np.fft.fft(wave_full[140000:])/len(wave_full[140000:])
plot(frq2[:int(len(fft_full)/2)],absolute(fft_full[:int(len(fft_full)/2)]))
plot(frq1[:int(len(fft_cal_geo)/2)],absolute(fft_cal_geo[:int(len(fft_cal_geo)/2)]))
print(len(fft_full),len(fft_cal_geo))

masking_geo = scipy.interpolate.interp1d(frq1,fft_cal_geo, kind='linear',fill_value='extrapolate')
fft_full_cal = masking_geo(frq2)
plot(frq2[:int(len(fft_full)/2)],absolute(fft_full_cal[:int(len(fft_full)/2)]))
reduc_fft = abs(fft_full)-abs(fft_full_cal)
plot(frq2[:int(len(fft_full)/2)],absolute(reduc_fft[:int(len(fft_full)/2)]))

xlim(0,60)

### Masking attempt

In [ ]:
import IPython
from scipy.io import wavfile
import scipy.signal
import numpy as np
import matplotlib.pyplot as plt
import librosa
%matplotlib inline

t= linspace(0,200,2000000)
t1= linspace(0,200,2000000)
t_fin = linspace(0,200,2000000)

t[:1000000] = 0
# t[199999:] = 0
wave1 = np.sin(10*2*np.pi*t)
# t1[:160000]= 0
# t1[179999:]= 0
# wave2 = np.sin(25*2*np.pi*t1)
freq_val = [10]#,25]
loc = [1999999]#,179999]
wave = wave1#+wave2
geo = 0.35*np.sin(2*2*np.pi*t_fin)+0.4*np.sin(3*2*np.pi*t_fin)+0.1*np.sin(1000*2*np.pi*t_fin)+0.3*np.sin(2.5*2*np.pi*t_fin)

transf_s = 0.5*np.ones(len(wave))
frq = np.fft.fftfreq(len(wave),1/10000)
transf_sample = scipy.interpolate.interp1d(frq,transf_s, kind='linear')

#Treating the steady state waveform as noise and removing it from the signal

# chirped_up,chirped_down = chirped_tip(wave,geo,t_fin,freq_val,loc,transf_sample)
# plot(t_fin,wave+geo)
# plot(t_fin,chirped_up)

# xlim(99,101)





In [ ]:
t = linspace(0,10,100000)
t1 = linspace(0,10,100000)
t[:40000] = 0
t[60000:] = 0
wave_part1 = np.sin(10*2*np.pi*t) + 0.333*np.sin(30*2*np.pi*t) + 0.2*np.sin(50*2*np.pi*t)
wave_fft = np.fft.fft(wave_part1)
wave_part2 = np.fft.ifft(wave_fft)
plot(abs(wave_fft[0:500]))

wave = np.sin(10*2*np.pi*t)
wave_fft2 = np.fft.fft(wave)
plot(abs(wave_fft2[0:500]))
# plot(t1,wave_part1)
# plot(t1,wave_part2)
# xlim(3.9,5)

In [ ]:
t= linspace(0,10,100000)
t1= linspace(0,10,100000)
t[:40000] = 0
t[70000:] = 0
wave = np.sin(10*2*np.pi*t)
freq_val = [10]
freq_loc = [69999]
prev_point = 39999
harmonics = arange(5)*2+3

amps = 1/harmonics
i = 0
for val_pnt,loc_pnt in zip(freq_val,freq_loc):

    chirp_part = np.zeros(len(wave))
#     bin_num = np.argmin(freqs-val_pnt)
    i=1
    if i == 0:
        chirp_part[0:loc_pnt]= wave[0:loc_pnt]
        i=1
    else:
        chirp_part[prev_point:loc_pnt] = wave[prev_point:loc_pnt]


    harm_freq = harmonics*val_pnt
    chirp_mag = amps*max(chirp_part)
    for freq,mag in zip(harm_freq,chirp_mag):

        chirp_part = chirp_part + mag*np.sin(freq*2*np.pi*t)
plot(t1,chirp_part)
xlim(3.9,5)
chirp = chirp_part

In [ ]:
>>> f, t, Sxx = signal.spectrogram(chirp, 10000)
>>> plt.pcolormesh(t,f, Sxx)
>>> plt.ylabel('Frequency [Hz]')
>>> plt.xlabel('Time [sec]')
>>> plt.show()

In [ ]:
freq_chirp = fft.fftfreq(len(chirp),1/10000)
fft_chirp = fft.fft(chirp)
plot(freq_chirp[:int(len(freq_chirp)/2)],(absolute(fft_chirp[:int(len(freq_chirp)/2)]))/len(fft_chirp))
xlim(0,100)

In [ ]:
period_num = 5
res = 1
final_freq = 300+res
freq_vec = linspace(res,final_freq,int((final_freq-res)/res))
# print(freq_vec)
# print(len(freq_vec))
t_step = int(1/res)
t_len = sum(period_num/freq_vec)
print(t_len)